<a href="https://colab.research.google.com/github/martin-kosiik/scraping-court-cases/blob/master/extract_foreign_courts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd 
import numpy as np

In [7]:
!pip install razdel

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
final_dataset = pd.read_csv('drive/My Drive/data/final_dataset_2.csv', encoding="UTF-8")

arbitrage_rulings_df = pd.read_csv("drive/My Drive/data/arbitrage_rulings.csv", encoding="UTF-8")
arbitrage_rulings_df['ruling_date'] = arbitrage_rulings_df.case_id.str.extract('__(\d{8})( *\(\d\))*$')[0]
arbitrage_rulings_df['ruling_date'] = pd.to_datetime(arbitrage_rulings_df['ruling_date'], format='%Y%m%d')
arbitrage_rulings_df['starting_year'] = arbitrage_rulings_df.case_id.str.extract('(\d{4})__\d{8} *(\(\d\))*$')[0]

arbitrage_rulings_df['case_number'] = arbitrage_rulings_df['court_id'] + '-' + arbitrage_rulings_df['case_spec_digit'].astype(int).astype(str) + '/' + \
                                        arbitrage_rulings_df['starting_year'].astype(int).astype(str)


key_diff = set(arbitrage_rulings_df['case_number']).intersection(final_dataset['Номер дела'])
where_diff = arbitrage_rulings_df['case_number'].isin(key_diff)
arbitrage_rulings_df = arbitrage_rulings_df[where_diff]


# Remove the the NEWPAGE token and the page number
arbitrage_rulings_df["proc_text"] = arbitrage_rulings_df.text.str.replace("NEWPAGE \n\d", "",
                                                                       regex=True)

arbitrage_rulings_df["proc_text"] = arbitrage_rulings_df.proc_text.str.replace("NEWPAGE", "")

In [19]:
from  razdel  import sentenize , tokenize
#from  navec  import   Navec 
def make_tokens(text):
  all_tokens_in_text  = []
  for sent in sentenize(text):
      tokens = [_.text  for  _  in  tokenize(sent.text)]
      all_tokens_in_text.append(tokens)
  return all_tokens_in_text


tokenized_rulings = [make_tokens(ruling) for ruling in arbitrage_rulings_df["proc_text"]]

In [5]:
import spacy
from spacy_stanza import StanzaLanguage


ModuleNotFoundError: ignored

In [6]:
!pip install stanza
!pip install fuzzymatcher

     |████████████████████████████████| 235kB 8.7MB/s 


In [7]:
!pip install spacy-stanza


In [8]:
!pip install pymorphy2==0.8

     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 7.1MB 8.1MB/s 


In [4]:
import stanza
from spacy_stanza import StanzaLanguage
from spacy import displacy
#stanza.download('ru')

ModuleNotFoundError: ignored

In [3]:
# https://stanfordnlp.github.io/stanza/tokenize.html
snlp = stanza.Pipeline(lang="ru", processors='tokenize,ner', tokenize_pretokenized=True))
#nlp = StanzaLanguage(snlp)
#nlp = 

NameError: ignored

In [11]:
text = """Андре́й Серге́евич Арша́вин (род. 29 мая 1981[4], Ленинград) — российский футболист, бывший капитан сборной России, заслуженный мастер спорта России (2008). Выступал на позициях атакующего полузащитника, второго нападающего, плеймейкера. При тренере Гусе Хиддинке был одним из основных игроков сборной России. В 2008 году после чемпионата Европы 2008 года попал в символическую сборную Европы по версии УЕФА, а 2 декабря, в результате голосования на определение лучшего игрока 2008 года и вручение награды «Золотой мяч» по версии журнала «France Football», занял 6-е место, что является лучшим результатом для российского футболиста после распада СССР. Неоднократно признавался лучшим футболистом России в различных опросах. С 5 декабря 2009 года входит в «Клуб 100 российских бомбардиров»."""
doc = nlp(text)


'<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Андре́й Серге́евич Арша́вин\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">PER</span>\n</mark>\n (род. 29 мая 1981[4], \n<mark class="entity" style="background: #ff9561; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Ленинград\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">LOC</span>\n</mark>\n) — российский футболист, бывший капитан \n<mark class="entity" style="background: #7aecec; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    сборной России\n    <span style="font-size: 0.8em

In [ ]:
stanza_docs_list = [snlp(text) for text in arbitrage_rulings_df["proc_text"]]

#text = arbitrage_rulings_df["proc_text"][2]
#doc = snlp(text)

In [2]:
stanza_docs_list

NameError: ignored

In [52]:
import re
captured_match = re.search('суд', list(doc.ents)[0].text, flags=re.IGNORECASE)

courts = [entity.text for entity in list(doc.ents) if re.search('суд', entity.text, flags=re.IGNORECASE) is not None]
print(captured_match.string)
courts
#list(doc.ents)

ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД


['ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД',
 'Тринадцатый арбитражный апелляционный суд',
 'Арбитражного  суда  Санкт-Петербурга',
 'Арбитражный  суд  Санкт-Петербурга']

In [68]:
this =list(doc.ents)[12]
this.

In [82]:
list(doc.ents)[12].text

'Арбитражного  суда  Санкт-Петербурга'

In [89]:
ent_list = [ent.text for ent in doc.entities]
courts = [entity for entity in ent_list if re.search('суд', entity, flags=re.IGNORECASE) is not None]
courts = [court.replace('\n','') for court in courts]
courts

['ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД',
 'Тринадцатый арбитражный апелляционный суд',
 'Арбитражного  суда  Санкт-Петербурга',
 'Арбитражный  суд  Санкт-Петербурга',
 'Тринадцатый  арбитражный апелляционный суд',
 'ПОСТАНОВИЛ: Решение  Арбитражного суда Санкт-Петербурга',
 'Федеральный  арбитражный  суд Северо-Западного округа']

In [105]:
russian_courts_df = pd.read_excel("drive/My Drive/data/russian_courts.xlsx")
russian_courts_df['russian_courts'] = russian_courts_df.russian_courts.str.replace('\s\(.+\)', '')

In [106]:
import fuzzymatcher
fuzzymatcher.fuzzy_left_join(pd.DataFrame(courts, columns=['entity']), russian_courts_df, left_on = "entity", right_on = "russian_courts")

,best_match_score,__id_left,__id_right,entity,russian_courts
0,0.177106,0_left,16_right,ТРИНАДЦАТЫЙ АРБИТРАЖНЫЙ АПЕЛЛЯЦИОННЫЙ СУД,Тринадцатый арбитражный апелляционный суд
1,0.177106,1_left,16_right,Тринадцатый арбитражный апелляционный суд,Тринадцатый арбитражный апелляционный суд
2,-0.144556,2_left,96_right,Арбитражного суда Санкт-Петербурга,Арбитражный суд Санкт-Петербурга и Ленинградск...
5,0.007809,3_left,96_right,Арбитражный суд Санкт-Петербурга,Арбитражный суд Санкт-Петербурга и Ленинградск...
6,0.177106,4_left,16_right,Тринадцатый арбитражный апелляционный суд,Тринадцатый арбитражный апелляционный суд
7,-0.244900,5_left,96_right,ПОСТАНОВИЛ: Решение Арбитражного суда Санкт-П...,Арбитражный суд Санкт-Петербурга и Ленинградск...
10,0.217344,6_left,8_right,Федеральный арбитражный суд Северо-Западного...,Арбитражный суд Северо-Западного округа
